In [4]:
import scanpy as sc
import os
import warnings
import anndata
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from scanpy.metrics.specificity.plot import marker_genes_distribution, one_v_max_genelist
import numpy as np
from scanpy.metrics.specificity.get_data import get_average_celltype_counts


In [5]:
# init functions
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

def find_max_celltype(gene,adata,partition_key="CellType"):
    get_average_celltype_counts(adata , partition_key = partition_key)
    if not np.where(adata.var.index == gene)[0] :
        max_celltype= None
    else :
        gene_index = int(np.where(adata.var.index == gene)[0])
        max_celltype_index = np.argmax(adata.varm['ave_celltype_counts_' + partition_key][gene_index,:])
        max_celltype = list(adata.obs[partition_key].cat.categories)[max_celltype_index]
    return(max_celltype)

def show_spec_gene(change):
    output_gene.clear_output()
    with output_gene :
        max_celltype = find_max_celltype(gene=change.new,adata=adata,partition_key="CellType")
        print('Specificity prediction : ' + str(specs.loc[change.new,'spe_type']) )
        print('Most represented celltype, predicted from specificity : ' + max_celltype )

        print('\n Discovair marker information :\n' + str(df_markers_aw.loc[change.new]))
        
        marker_genes_distribution(adata=adata,
                                  gene_list=[change.new],
                                  celltype=max_celltype,
                                  partition_key='CellType')
        one_v_max_genelist(adata=adata,
                           gene_list=[change.new],
                           partition_key='CellType')
        sc.pl.umap(adata=adata, color=change.new, color_map = 'jet')
def show_spec_distrib(change):
    output_marker.clear_output()
    with output_marker :
        spec_markers=markers[change.new].dropna()
        found_markers=spec_markers[spec_markers.isin(adata.var_names)]
        not_found=spec_markers[~spec_markers.isin(adata.var_names)]
        if not not_found.empty:
            print('Gene(s) not found : ' + ','.join(not_found))
        print(specs.loc[found_markers,'spe_type'])
        marker_genes_distribution(adata=adata,
                                  gene_list=spec_markers,
                                  celltype=change.new,
                                  partition_key='CellType')
        one_v_max_genelist(adata=adata,
                           gene_list=spec_markers,
                           partition_key='CellType')
        sc.pl.umap(adata=adata, color=found_markers, color_map = 'jet')

## Load data HCA Barbry

In [6]:
DATA_PATH = 'data/'

In [7]:
count_file = 'HCA_Barbry_Grch38_Raw_filter_Norm.h5ad'
adata = anndata.read_h5ad(DATA_PATH + count_file)

In [8]:
adata.obs['CellType'].cat.categories

Index(['AT1', 'AT2', 'B cells', 'Basal', 'Brush cells', 'Cycling Basal',
       'Dendritic', 'Deuterosomal', 'Endothelial', 'Fibroblast', 'Ionocyte',
       'LT/NK', 'Macrophage', 'Mast cells', 'Monocyte', 'Multiciliated',
       'Multiciliated N', 'PNEC', 'Pericyte', 'Plasma cells', 'Precursor',
       'SMG Goblet', 'Secretory', 'Secretory N', 'Serous', 'Smooth muscle',
       'Suprabasal', 'Suprabasal N'],
      dtype='object')

In [9]:
specs = pd.read_csv(DATA_PATH + 'genes_spec_pred_V1', index_col = 'gene')

## Load marker tables

In [10]:
df_markers_aw = pd.read_csv(DATA_PATH + 'SCRINSHOT_Airwaywall_probeset.csv', index_col = 'gene')
df_markers_aw = df_markers_aw[df_markers_aw['selection'] == True]
df_markers_aw.drop('Unnamed: 0',1)

,selection_ranking,selection,AW/Par,pca_gene,tree_gene,list_marker,marker
gene,,,,,,,
HLA-DRA,1,True,shared,True,"Myeloid,Mast cells,Pericytes,Deuterosomal,Muco...",none,Myeloid
AREG,1,True,shared,True,"Club,Mast cells",none,Mast cells
IFITM1,1,True,shared,True,"(Vascular) Endothelial cell,Lymphatic EC",none,(Vascular) Endothelial cell
CST3,1,True,shared,True,"Lymphocyte,Myeloid,Chondrocytes",none,"Chondrocytes,Myeloid"
S100A4,1,True,shared,True,"(Vascular) Endothelial cell,Basal,Myeloid",none,Myeloid
S100A9,1,True,shared,True,Myeloid,none,Myeloid
VIM,1,True,shared,True,"Basal,Club,Lymphatic EC",none,Lymphatic EC
MT1A,1,True,shared,True,Pericytes,none,Pericytes
TPSAB1,1,True,shared,True,Mast cells,none,Mast cells


In [11]:
df_markers_par = pd.read_csv(DATA_PATH + 'SCRINSHOT_Parenchyma_probeset.csv', index_col = 'gene')
df_markers_par = df_markers_par[df_markers_par['selection'] == True]
df_markers_par.drop('Unnamed: 0',1)

,selection_ranking,selection,AW/Par,pca_gene,tree_gene,list_marker,marker
gene,,,,,,,
IL7R,1,True,shared,True,"(Vascular) Endothelial cell,Cappillary (G) end...",none,Cappillary (G) endothelial cell
S100A4,1,True,shared,True,"(Vascular) Endothelial cell,Basal,Myeloid,Capp...",none,Myeloid
S100A9,1,True,shared,True,Myeloid,none,Myeloid
MT1A,1,True,shared,True,Pericytes,none,Pericytes
IFITM1,1,True,shared,True,"(Vascular) Endothelial cell,Lymphatic EC",none,(Vascular) Endothelial cell
CXCR4,1,True,shared,True,"Lymphocyte,Mast cells",none,Lymphocyte
HLA-DRA,1,True,shared,True,"Myeloid,Mast cells,Pericytes",none,Myeloid
FOS,1,True,shared,True,Basal,none,Basal
TIMP1,1,True,shared,True,"Adventitial Fibroblast,Mesothelial cell",none,"Adventitial Fibroblast,Mesothelial cell"


# Display Prediction on marker from Airway wall

Specificity prediction are in 5 categories : equirep low multirep high unique

In [ ]:
combobox_genes = widgets.Combobox(
    options=list(df_markers_aw.index),
    placeholder='Type a gene',
    description='Gene:',
    ensure_option=True,
    disabled=False,
)
output_gene=widgets.Output()
combobox_genes.observe(show_spec_gene,names='value')
display(combobox_genes)
display(output_gene)

# Display Prediction on marker from Parenchyma

In [ ]:
combobox_genes = widgets.Combobox(
    options=list(df_markers_par.index),
    placeholder='Type a gene',
    description='Gene:',
    ensure_option=True,
    disabled=False,
)
output_gene=widgets.Output()
combobox_genes.observe(show_spec_gene,names='value')
display(combobox_genes)
display(output_gene)

### Save specificity prediction in a table

In [ ]:
# Find airway wall marker gene not in our dataset
print("Marker genes not present in dataset :",df_markers_aw.index.difference(specs.index))

In [ ]:
list_markers = [x for x in df_markers_aw.index if x in specs.index]
df_markers_aw.loc[list_markers,"Specificity_Barbry"] = specs.loc[list_markers,"spe_type"]
list_celltype = [find_max_celltype(gene,adata,partition_key="CellType") for gene in list_markers]
df_markers_aw.loc[list_markers,"Cell_type_Spe_Barbry"] = list_celltype

In [ ]:
# Find parenchyma marker gene not in our dataset
print("Marker genes not present in dataset :",df_markers_par.index.difference(specs.index))

In [ ]:
list_markers = [x for x in df_markers_par.index if x in specs.index]
df_markers_par.loc[list_markers,"Specificity_Barbry"] = specs.loc[list_markers,"spe_type"]
list_celltype = [find_max_celltype(gene,adata,partition_key="CellType") for gene in list_markers]
df_markers_par.loc[list_markers,"Cell_type_Spe_Barbry"] = list_celltype

## Load data HCA Meyer

In [ ]:
df_meyer = pd.read_csv(DATA_PATH + 'Gene_Meyer_Barbry_comparison.txt', sep='\t', index_col = 'Genes')

In [ ]:
list_markers = [x for x in df_markers_aw.index if x in df_meyer.index]
list_spe = list(df_meyer.loc[list_markers,"spe_type_Meyer"])
df_markers_aw.drop('Unnamed: 0',1)
df_markers_aw.loc[list_markers,"Spqdqzecificity_Meyer"] = list_spe
#df_markers_aw.loc[list_markers,"Ceqzdll_type_Spe"] = df_meyer.loc[list_markers,"CellType"]
df_markers_aw.to_csv(DATA_PATH + 'SCRINSHOT_Airwaywall_probeset_specificity.csv')

In [ ]:
list_markers = [x for x in df_markers_par.index if x in df_meyer.index]
#df_markers_par.loc[list_markers,"Specificity_Meyer"] = df_meyer.loc[list_markers,"spe_type_Meyer"]
#df_markers_par.loc[list_markers,"Cell_type_Spe"] = df_meyer.loc[list_markers,"CellType"]
df_markers_par.to_csv(DATA_PATH + 'SCRINSHOT_Parenchyma_probeset_specificity.csv')

## Temporary spe prediction for SCRINSHOT probes (not very pretty)

In [12]:
df_meyer_barbry = pd.read_csv('spec_all_genes_Barbry_Meyer.csv', sep=',', index_col = 'gene')

In [13]:
df_meyer_barbry.columns = df_meyer_barbry.columns[4,9,0,1,2,3,5,6,7,8]

IndexError: too many indices for array: array is 1-dimensional, but 10 were indexed

In [16]:
df_meyer_barbry = df_meyer_barbry[['spe_type_Barbry','spe_type_Meyer','one_v_max_Barbry','shannon_Barbry','tau_Barbry','gini_Barbry','one_v_max_Meyer','shannon_Meyer','tau_Meyer','gini_Meyer']]
meyer_barbry_spe = df_meyer_barbry[['spe_type_Barbry','spe_type_Meyer']]

In [17]:
df_markers_aw_spe = df_markers_aw.join(meyer_barbry_spe)
df_markers_par_spe = df_markers_par.join(meyer_barbry_spe)

In [18]:
count_file_barbry = 'HCA_Barbry_Grch38_Raw_filter_Norm.h5ad'
adata_barbry = anndata.read_h5ad(DATA_PATH + count_file_barbry)

count_file_meyer = 'lung.cellxgene.h5ad'
adata_meyer = anndata.read_h5ad(DATA_PATH + count_file_meyer)

for gene in df_markers_aw_spe.index :
    max_celltype_barbry = find_max_celltype(gene=gene,adata=adata_barbry)
    max_celltype_meyer = find_max_celltype(gene=gene,adata=adata_meyer,partition_key='Celltypes_updated_July_2020')
    df_markers_aw_spe.loc[gene,'max_celltype_barbry'] = max_celltype_barbry
    df_markers_aw_spe.loc[gene,'max_celltype_meyer'] = max_celltype_meyer
    
for gene in df_markers_par_spe.index :
    max_celltype_barbry = find_max_celltype(gene=gene,adata=adata_barbry)
    max_celltype_meyer = find_max_celltype(gene=gene,adata=adata_meyer,partition_key='Celltypes_updated_July_2020')
    df_markers_par_spe.loc[gene,'max_celltype_barbry'] = max_celltype_barbry
    df_markers_par_spe.loc[gene,'max_celltype_meyer'] = max_celltype_meyer
    

In [19]:
print(len(df_markers_aw) - len(df_markers_aw_spe))
print(len(df_markers_par) - len(df_markers_par_spe))

0
0


In [22]:
df_markers_par_spe = df_markers_par_spe[['Unnamed: 0', 'selection_ranking', 'selection', 'AW/Par', 'pca_gene',
       'tree_gene', 'list_marker', 'marker', 'spe_type_Barbry',
       'spe_type_Meyer', 'max_celltype_barbry', 'max_celltype_meyer']]

df_markers_aw_spe = df_markers_aw_spe[['Unnamed: 0', 'selection_ranking', 'selection', 'AW/Par', 'pca_gene',
       'tree_gene', 'list_marker', 'marker', 'spe_type_Barbry',
       'spe_type_Meyer','max_celltype_barbry', 'max_celltype_meyer']]

In [23]:
df_markers_aw_spe.to_csv('SCRINSHOT_AW_Wall_Specificity_Meyer_Barbry.csv')
df_markers_par_spe.to_csv('SCRINSHOT_AW_Parenchyme_Specificity_Meyer_Barbry.csv')